In [ ]:

import pandas as pd
import numpy as np
import datetime
from scipy.optimize import newton

filename = "../data/transactions.csv"

usd_to_ars = 1000
current_prices = {
    "AAPL": 14025.0,
    "SPY": 34725.0,
    "BRKB": 25550.0,
    "EEM": 10900.0,
    "IWM": 26775.0,
    "NVDA": 5690.0,
    "QQQ": 29325.0
}
stock_splits = {
    "MMM": [(pd.to_datetime("2024-01-24"), 2)],
    "ADGO": [(pd.to_datetime("2024-01-24"), 0.5)],
    "ADBE": [(pd.to_datetime("2024-01-24"), 2)],
    "AEM": [(pd.to_datetime("2024-01-24"), 2)],
    "AMGN": [(pd.to_datetime("2024-01-24"), 3)],
    "AAPL": [(pd.to_datetime("2024-01-24"), 2)],
    "BAC": [(pd.to_datetime("2024-01-24"), 2)],
    "GOLD": [(pd.to_datetime("2024-01-24"), 2)],
    "BIOX": [(pd.to_datetime("2024-01-24"), 0.5)],
    "CVX": [(pd.to_datetime("2024-01-24"), 2)],
    "LLY": [(pd.to_datetime("2024-01-24"), 7)],
    "XOM": [(pd.to_datetime("2024-01-24"), 2)],
    "FSLR": [(pd.to_datetime("2024-01-24"), 6)],
    "IBM": [(pd.to_datetime("2024-01-24"), 3)],
    "JD": [(pd.to_datetime("2024-01-24"), 2)],
    "JPM": [(pd.to_datetime("2024-01-24"), 3)],
    "MELI": [(pd.to_datetime("2024-01-24"), 2)],
    "NFLX": [(pd.to_datetime("2024-01-24"), 3)],
    "PEP": [(pd.to_datetime("2024-01-24"), 3)],
    "PFE": [(pd.to_datetime("2024-01-24"), 2)],
    "PG": [(pd.to_datetime("2024-01-24"), 3)],
    "RIO": [(pd.to_datetime("2024-01-24"), 2)],
    "SONY": [(pd.to_datetime("2024-01-24"), 2)],
    "SBUX": [(pd.to_datetime("2024-01-24"), 3)],
    "TXR": [(pd.to_datetime("2024-01-24"), 2)],
    "BA": [(pd.to_datetime("2024-01-24"), 4)],
    "TM": [(pd.to_datetime("2024-01-24"), 3)],
    "VZ": [(pd.to_datetime("2024-01-24"), 2)],
    "VIST": [(pd.to_datetime("2024-01-24"), 3)],
    "WMT": [(pd.to_datetime("2024-01-24"), 3)],
    "NVDA": [(pd.to_datetime("2024-06-06"), 10)]
}

def adjust_for_splits(row):
    """Adjust for CEDEAR splits"""
    symbol = row['simbolo']
    date = row['fechaOperada']
    cantidad = row['cantidadOperada']
    precio = row['precioOperado']
    
    if symbol in stock_splits:
        for split_date, ratio in stock_splits[symbol]:
            if date < split_date:
                cantidad *= ratio
                if pd.notna(precio):
                    precio /= ratio
                
    return pd.Series({'cantidadOperada': cantidad, 'precioOperado': precio})

def calculate_irr(cash_flows):
    cash_flows = [cf for cf in cash_flows if not pd.isna(cf[1])]
    cash_flows = sorted(cash_flows, key=lambda x: x[0])
    values = np.array([cf[1] for cf in cash_flows])

    def npv(rate):
        return sum(cf / (1 + rate) ** i for i, cf in enumerate(values))
    
    try:
        return newton(npv, 0.1)
    except RuntimeError:
        return np.nan

def process_investment_data(filename):
    df = pd.read_csv(filename)
    df = df[df["estado"] == "terminada"]

    # Convert 'fechaOperada' to datetime using mixed format
    df['fechaOperada'] = pd.to_datetime(df['fechaOperada'], format='mixed')

    # Identify USD transactions and normalize ticker names
    df["is_usd"] = df["simbolo"].str.contains("US$")
    df['simbolo'] = df['simbolo'].str.replace(' US$', '', regex=False)

    df['tipo'] = df['tipo'].replace({'Suscripción FCI': 'Compra', 'Rescate FCI': 'Venta'})
    df['montoOperado'] = df.apply(
        lambda row: row['montoOperado'] * usd_to_ars if row['is_usd'] and row['tipo'] == 'Pago de Dividendos' else row['montoOperado'],
        axis=1
    )

    df[['cantidadOperada', 'precioOperado']] = df.apply(adjust_for_splits, axis=1)

    grouped = df.groupby('simbolo')

    # Calculate financial metrics
    result = grouped.apply(lambda x: pd.Series({
        'total_cost': x.loc[x['tipo'] == 'Compra', 'montoOperado'].sum(),
        'total_dividends': x.loc[x['tipo'] == 'Pago de Dividendos', 'montoOperado'].sum(),
        'total_sales': x.loc[x['tipo'] == 'Venta', 'montoOperado'].sum(),
        'total_proceeds': x.loc[x['tipo'].isin(['Venta', 'Pago de Dividendos']), 'montoOperado'].sum(),
        'current_quantity': x.loc[x['tipo'] == 'Compra', 'cantidadOperada'].sum() - x.loc[x['tipo'] == 'Venta', 'cantidadOperada'].sum(),
        'holding_period_years': (datetime.datetime.now() - x['fechaOperada'].min()).days / 365.0,
        'min_date': x['fechaOperada'].min()
    })).reset_index()

    result = result[result["simbolo"].isin(current_prices.keys())]
    result['current_price'] = result['simbolo'].map(current_prices)
    result['holding_value'] = result['current_price'] * result['current_quantity']
    result["net_profit_loss"] = result["total_proceeds"] - result["total_cost"]

    # Calculate CAGR (compound annual growth rate)
    result["cagr"] = (
        ((result["total_proceeds"] + result["holding_value"]) / result["total_cost"]) ** (1 / result["holding_period_years"])
    ) - 1

    # Calculate IRR
    cash_flows = {symbol: [] for symbol in result['simbolo']}
    # Add sales and purchases as cash flows
    for idx, row in df.iterrows():
        symbol = row['simbolo']
        date = row['fechaOperada']
        amount = row['montoOperado']
        
        if symbol in cash_flows:
            cash_flows[symbol].append((date, -amount if row['tipo'] == 'Compra' else amount))

    for symbol in result['simbolo']:
        holding_value = result.loc[result['simbolo'] == symbol, 'holding_value'].values[0]
        cash_flows[symbol].append((datetime.datetime.now(), holding_value))
        result.loc[result['simbolo'] == symbol, 'irr'] = calculate_irr(cash_flows[symbol])

    return result

result = process_investment_data('../data/transactions.csv')
result
